In [ ]:
%load_ext autoreload
%autoreload 2

# Test Checkpoint

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"

unet = UNet2DConditionModel.from_pretrained(
    "/data/finetune/t2i/living_room/20230615-012615/checkpoint-500/unet",
    torch_dtype=torch.float16,
)

# if you have trained with `--args.train_text_encoder` make sure to also load the text encoder
#text_encoder = CLIPTextModel.from_pretrained("/sddata/dreambooth/daruma-v2-1/checkpoint-100/text_encoder")

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    unet=unet,
    safety_checker=None,
    torch_dtype=torch.float16
)
pipe.to("cuda")

In [ ]:
#prompt = "interior design, living room, brown leather couch, high res, 4k"
prompt = "Yoda"
#prompt = "man riding a horse"
seed = 8


In [ ]:
generator = torch.Generator(device="cuda").manual_seed(seed)

image = pipe(
    prompt=prompt,
    generator=generator,
    num_inference_steps=50,
).images[0]
image

# Test Merged Checkpoint

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from utils.merge_weights import merge

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    safety_checker=None,
    torch_dtype=torch.float16
)

In [ ]:
unet = UNet2DConditionModel.from_pretrained(
    "/data/finetune/t2i/living_room/20230615-012615/checkpoint-1000/unet",
    torch_dtype=torch.float16,
)

In [ ]:
merged_unet = merge(pipe.unet, unet, 0.5)

In [ ]:
pipe.to("cuda")

In [ ]:
#prompt = "interior design, living room, brown leather couch, high res, 4k"
prompt = "Yoda"
#prompt = "man riding a horse"
seed = 4

In [ ]:
generator = torch.Generator(device="cuda").manual_seed(seed)

image = pipe(
    prompt=prompt,
    generator=generator,
    num_inference_steps=50,
).images[0]
image